# Домашнее задание №2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('./webinar_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
data_train[['user_id', 'sales_value']]

,user_id,sales_value
0,2375,1.39
1,2375,0.82
2,2375,0.99
3,2375,1.21
4,2375,1.50
...,...,...
2282320,222,0.59
2282321,462,1.99
2282322,462,1.00
2282323,462,3.00


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])    
    return recs.tolist()

In [6]:
%%time

# your_code

popular_sum = data.groupby('item_id')['sales_value'].sum().reset_index()
popular_sum['weight'] = popular_sum['sales_value']/popular_sum['sales_value'].sum()

rsult = weighted_random_recommendation(popular_sum.drop(columns=['sales_value']), n=5)
rsult1

NameError: name 'rsult1' is not defined

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [8]:
result = pd.read_csv('./webinar_2/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[846767, 982824, 6442817, 995840, 9706823]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1121213, 808299, 12346966, 13416093, 1044599]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [9]:
rc_list=result.columns[2:]
argk=5

rc_list, argk

(Index(['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine',
        'tfidf', 'own_purchases'],
       dtype='object'),
 5)

In [10]:
# your_code
for name_col in rc_list:
    prs = result.apply(lambda x: precision_at_k(recommended_list=result[name_col], bought_list=result['actual'], k=argk), axis=1)
    print(f"precision_at_k for {name_col} is {round(prs.mean(), 5)}")

precision_at_k for random_recommendation is 0.0
precision_at_k for popular_recommendation is 0.0
precision_at_k for itemitem is 0.0
precision_at_k for cosine is 0.0
precision_at_k for tfidf is 0.0
precision_at_k for own_purchases is 0.0


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [11]:
# your_code